In [ ]:
import pandas as pd

In [ ]:
cols = [x for x in range(1,26)]

In [ ]:
matches_df = pd.read_csv('matches_no_doup.csv')

In [ ]:
matches_df['opponent'] = matches_df['opponent'].str.split().str[1:].str.join(' ')

In [ ]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>



RangeIndex: 1391 entries, 0 to 1390



Data columns (total 25 columns):



 #   Column        Non-Null Count  Dtype  



---  ------        --------------  -----  



 0   date          1391 non-null   object 



 1   opponent      1391 non-null   object 



 2   team          1391 non-null   object 



 3   poss          1369 non-null   float64



 4   sh            1370 non-null   float64



 5   sot           1370 non-null   float64



 6   gf            1390 non-null   object 



 7   ga            1390 non-null   object 



 8   time          1391 non-null   object 



 9   comp          1391 non-null   object 



 10  round         1391 non-null   object 



 11  day           1391 non-null   object 



 12  venue         1391 non-null   object 



 13  result        1390 non-null   object 



 14  attendance    693 non-null    float64



 15  captain       1292 non-null   object 



 16  formation     1287 non-null   object 



 17  refer

In [ ]:
matches_df['date'] = pd.to_datetime(matches_df['date'])

In [ ]:
#clening the gf and ga column and making it an int
matches_df['gf'] = matches_df['gf'].str.split().str[0]
matches_df['gf'] = matches_df['gf'].str.split('.').str[0]

matches_df['ga'] = matches_df['ga'].str.split().str[0]
matches_df['ga'] = matches_df['ga'].str.split('.').str[0]

matches_df.dropna(subset=['gf'], inplace=True)
matches_df['gf'] = matches_df['gf'].astype(int)
matches_df['ga'] = matches_df['ga'].astype(int)

In [ ]:
matches_df['opp_code'] = matches_df.opponent.astype('category').cat.codes 
matches_df['team_code'] = matches_df.team.astype('category').cat.codes 

In [ ]:
matches_df['hour'] = matches_df['time'].str.replace(':.+', '', regex=True).astype(int)

In [ ]:
matches_df['round_code'] = matches_df['round'].astype('category').cat.codes 

In [ ]:
matches_df['day_code'] = matches_df['date'].dt.dayofweek

In [ ]:
#matches_df['target'] = matches_df['result'].astype('category').cat.codes
matches_df['target'] = (matches_df['result'] == 'W').astype(int)

In [ ]:
matches_df.head()

,date,opponent,team,poss,sh,sot,gf,ga,time,comp,...,pkatt,season,xg,xga,opp_code,team_code,hour,round_code,day_code,target
0,2020-10-08,Argentina,Ecuador,42.0,4.0,0.0,0,1,21:30,WCQ,...,0.0,2022,NaN,NaN,5,9,21,14,3,0
1,2020-10-13,Uruguay,Ecuador,51.0,17.0,7.0,4,2,16:00,WCQ,...,0.0,2022,NaN,NaN,163,9,16,14,1,1
2,2020-11-12,Bolivia,Ecuador,62.0,12.0,6.0,3,2,16:00,WCQ,...,1.0,2022,NaN,NaN,18,9,16,14,3,1
3,2020-11-17,Colombia,Ecuador,53.0,15.0,10.0,6,1,16:00,WCQ,...,0.0,2022,NaN,NaN,34,9,16,14,1,1
4,2021-06-04,Brazil,Ecuador,34.0,3.0,0.0,0,2,21:30,WCQ,...,0.0,2022,NaN,NaN,21,9,21,14,4,0


In [ ]:
matches_df.dtypes

date            datetime64[ns]
opponent                object
team                    object
poss                   float64
sh                     float64
sot                    float64
gf                       int64
ga                       int64
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                   object
pk                     float64
pkatt                  float64
season                  object
xg                     float64
xga                    float64
opp_code                 int16
team_code                 int8
hour                     int64
round_code                int8
day_code                 int64
target                   int64
dtype: object

In [ ]:
matches_no_na = matches_df.dropna(subset=['poss','sh', 'sot'])
print('len: ', len(matches_no_na['team'].value_counts()))
matches_no_na['team'].value_counts()


len:  32


Brazil            74
Argentina         70
Uruguay           64
Australia         62
Ecuador           60
Mexico            50
United-States     45
Tunisia           43
Japan             42
Spain             42
Qatar             41
Belgium           41
England           41
Morocco           40
France            40
Denmark           40
Senegal           40
Wales             39
Poland            39
Portugal          38
Switzerland       38
Croatia           38
Netherlands       38
Germany           38
Serbia            37
Cameroon          37
Costa-Rica        36
Ghana             32
Korea-Republic    32
Canada            31
Saudi-Arabia      31
IR-Iran           30
Name: team, dtype: int64

In [ ]:
matches_no_na['g/sh'] = matches_no_na['gf'] / matches_no_na['sh']
matches_no_na['g/sot'] = matches_no_na['gf'] / matches_no_na['sot']

In [ ]:
matches_no_na['g/sh']  = matches_no_na['g/sh'].fillna(0)
matches_no_na['g/sot']  = matches_no_na['g/sot'].fillna(0)

In [ ]:
matches_no_na.drop(list(matches_no_na[matches_no_na['g/sh'] > 1].index), inplace=True)

In [ ]:
matches_no_na['g/sh'].isna().sum()

0

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
train = matches_no_na[matches_no_na['date'] < '2022-01-01']

In [ ]:
test = matches_no_na[matches_no_na['date'] > '2022-01-01']

In [ ]:
print(len(train) ,len(test))

1149 215


In [ ]:
predictors = ['opp_code', 'hour', 'day_code', 'round_code']

In [ ]:
X_train = train[predictors]
y_train = train['target']

In [ ]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [ ]:
acc = accuracy_score(test['target'], preds)

In [ ]:
acc

0.49767441860465117

In [ ]:
combined = pd.DataFrame(dict(actual = test['target'], prediction = preds))

In [ ]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0,1
actual,,
0,45,56
1,52,62


In [ ]:
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, train[predictors],  train['target'], cv=10)
scores

array([0.5826087 , 0.5826087 , 0.5826087 , 0.5826087 , 0.59130435,
       0.59130435, 0.59130435, 0.59130435, 0.59130435, 0.5877193 ])

In [ ]:
import xgboost

In [ ]:
model_xgboost = xgboost.XGBClassifier(learning_rate=0.1,
                                     max_depth=5,
                                     n_estimators=5000,
                                     subsample=0.5,
                                     colsample_bytree=0.5,
                                     eval_metric = 'auc')

eval_set = [(test[predictors], test['target'])]

model_xgboost.fit(X_train,
                 y_train,
                 eval_set=eval_set,
                 early_stopping_rounds=10)

[0]	validation_0-auc:0.55311



[1]	validation_0-auc:0.51663



[2]	validation_0-auc:0.53022



[3]	validation_0-auc:0.53630



[4]	validation_0-auc:0.54399



[5]	validation_0-auc:0.57009



[6]	validation_0-auc:0.57369



[7]	validation_0-auc:0.56618



[8]	validation_0-auc:0.57300



[9]	validation_0-auc:0.56080



[10]	validation_0-auc:0.56262



[11]	validation_0-auc:0.57304



[12]	validation_0-auc:0.57017



[13]	validation_0-auc:0.58199



[14]	validation_0-auc:0.58086



[15]	validation_0-auc:0.57230



[16]	validation_0-auc:0.57760



[17]	validation_0-auc:0.58359



[18]	validation_0-auc:0.57925



[19]	validation_0-auc:0.57769



[20]	validation_0-auc:0.57148



[21]	validation_0-auc:0.57513



[22]	validation_0-auc:0.57513



[23]	validation_0-auc:0.57556



[24]	validation_0-auc:0.58099



[25]	validation_0-auc:0.56462



[26]	validation_0-auc:0.56088


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=5000, n_jobs=16,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.5, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_train_proba = model_xgboost.predict_proba(X_train)[:,1]
y_test_proba = model_xgboost.predict_proba(test[predictors])[:,1]
print(f'AUC Train: {roc_auc_score(y_train, y_train_proba)}\nAUC Valid: {roc_auc_score(test.target ,y_test_proba)}')

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(test['target'], preds)

In [ ]:
grouped_matches = matches_no_na.groupby('team')

In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(5, closed ='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols = ['gf', 'ga', 'sh', 'sot', 'poss', 'pk', 'pkatt', 'g/sh', 'g/sot']
new_cols = [f'{c}_rolling' for c in cols]

In [ ]:
matches_rolling = matches_no_na.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling.head()

In [ ]:
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling.info()

In [ ]:
def make_predictions(data, predictors):
    train = data[data['date'] < '2022-01-01']
    test = data[data['date'] > '2022-01-01']
    rf.fit(train[predictors], train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test['target'], prediction = preds))
    precision = precision_score(test['target'], preds)
    return combined, precision

In [ ]:
train = matches_rolling[matches_rolling['date'] < '2022-01-01']
test = matches_rolling[matches_rolling['date'] > '2022-01-01']

model_xgboost = xgboost.XGBClassifier(learning_rate=0.1,
                                     max_depth=5,
                                     n_estimators=5000,
                                     subsample=0.5,
                                     colsample_bytree=0.5,
                                     eval_metric = 'auc')

eval_set = [(test[predictors], test['target'])]

model_xgboost.fit(train[predictors],
                 train['target'],
                 eval_set=eval_set,
                 early_stopping_rounds=10)   

y_train_pred = model_xgboost.predict_proba(train[predictors])[:,1]
y_test_proba = model_xgboost.predict_proba(test[predictors])[:,1]
print(f'AUC Train: {roc_auc_score(y_train, y_train_proba)}\nAUC Valid: {roc_auc_score(test.target ,y_test_proba)}')

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
precision

In [ ]:
combined = combined.merge(matches_rolling[['date', 'team', 'opponent', 'result']], left_index=True, right_index=True)

In [ ]:
combined

In [ ]:
merged = combined.merge(combined, left_on=['date', 'team'], right_on=['date', 'opponent'])

In [ ]:
merged

<h1>Hyper Parameter Tunning</h1>

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
learning_rate_list = [0.02, 0.05, 0.1, 0.15]
max_depth_list = [2, 3, 5, 7]
n_estimator_list = [1000, 2000, 3000, 4000]

param_dict = {'learning_rate': learning_rate_list,
              'max_depth': max_depth_list,
              'n_estimators': n_estimator_list}
param_dict

In [ ]:
def my_roc_auc_score(model, X, y): return roc_auc_score(y, model.predict_proba(X)[:,1])

model_xgboost_hp = GridSearchCV(estimator=xgboost.XGBClassifier(subsample=0.5,
                                                                colsample_bytree=0.5,
                                                                eval_metric = 'auc',
                                                                use_label_encoder=False),
                               param_grid=param_dict,
                               cv=2,
                               scoring=my_roc_auc_score,
                               return_train_score=True,
                               verbose=4,
                               error_score='raise')

model_xgboost_hp.fit(matches_rolling[predictors + new_cols], matches_rolling['target'])

Fitting 2 folds for each of 64 candidates, totalling 128 fits

[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=1000;, score=(train=0.926, test=0.571) total time=   5.1s

[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=1000;, score=(train=0.913, test=0.560) total time=   2.8s

[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=2000;, score=(train=0.971, test=0.561) total time=   7.0s

[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=2000;, score=(train=0.965, test=0.564) total time=   5.7s

[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=3000;, score=(train=0.990, test=0.562) total time=  11.3s

[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=3000;, score=(train=0.985, test=0.564) total time=  11.8s

[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=4000;, score=(train=0.996, test=0.558) total time=  13.0s

[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=4000;, score=(train=0.995, test=0.570) total time=  

GridSearchCV(cv=2, error_score='raise',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.5, eval_metric='auc',
                                     gamma=None, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=0.5,
                                     tree_method=None, use

In [ ]:
df_cv_results = pd.DataFrame(model_xgboost_hp.cv_results_) 
df_cv_results = df_cv_results[['rank_test_score', 'mean_test_score', 'mean_train_score',
                              'param_learning_rate', 'param_max_depth', 'param_n_estimators']]
df_cv_results.sort_values(by='rank_test_score', inplace=True)
df_cv_results

,rank_test_score,mean_test_score,mean_train_score,param_learning_rate,param_max_depth,param_n_estimators
0,1,0.565541,0.919116,0.02,2,1000
16,2,0.565256,0.978337,0.05,2,1000
4,3,0.564907,0.988268,0.02,3,1000
18,4,0.563745,0.999959,0.05,2,3000
3,5,0.563560,0.995460,0.02,2,4000
...,...,...,...,...,...,...
47,60,0.537667,1.000000,0.1,7,4000
46,61,0.537530,1.000000,0.1,7,3000
45,62,0.536770,1.000000,0.1,7,2000
60,63,0.536141,1.000000,0.15,7,1000
